<a href="https://colab.research.google.com/github/ishumann/Coding-Implementation/blob/main/CNNOptimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

print(tf.__version__)

fashion_mnist= keras.datasets.fashion_mnist

(train_images,train_labels), (test_images,test_labels) = fashion_mnist.load_data()

train_images = train_images/255.0
test_images = test_images/255.0


def build_model(hp):
    model = keras.Sequential([
        keras.layers.Conv2D(
            filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16 ),
            activation='relu',
            kernel_size= hp.Choice('con_1_kernal', values=[3,5,7]),
            input_shape= (28,28,1)
        ),

        keras.layers.Conv2D(
            filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16 ),
            activation='relu',
            kernel_size= hp.Choice('con_1_kernal', values=[3,5,7])
        ),

        keras.layers.Flatten(),
        keras.layers.Dense(
            units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16 ),
            activation='relu'
        ),
        keras.layers.Dense(units= 10, activation='softmax')
    ])

    model.compile(optimizer=
                  keras.optimizers.Adam(hp.Choice('learning_rate',
                                                  values=[1e-2,1e-3])),
                  loss= "sparse_categorical_crossentropy", metrics= ['accuracy'] )
    return model

2.15.0


In [ ]:
import keras_tuner as kt
from keras_tuner.tuners import RandomSearch
from keras_tuner import HyperParameters

In [ ]:
tuner_search = RandomSearch(build_model,
#                             direction="min",
                            objective='val_accuracy',
                            max_trials=5,
                            directory='output',
                            project_name='Mnist Fashion')
# keras_tuner.Objective("val_accuray", direction="min")

In [ ]:
tuner_search.search(train_images, train_labels, epochs=2, validation_split=0.1)

Trial 5 Complete [00h 00m 17s]
val_accuracy: 0.8306666612625122

Best val_accuracy So Far: 0.8930000066757202
Total elapsed time: 00h 01m 44s


In [ ]:
tuner_search.results_summary()

Results summary
Results in output/Mnist Fashion
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 1 summary
Hyperparameters:
conv_1_filter: 64
con_1_kernal: 7
dense_1_units: 128
learning_rate: 0.001
Score: 0.8930000066757202

Trial 3 summary
Hyperparameters:
conv_1_filter: 64
con_1_kernal: 7
dense_1_units: 48
learning_rate: 0.001
Score: 0.8895000219345093

Trial 2 summary
Hyperparameters:
conv_1_filter: 112
con_1_kernal: 5
dense_1_units: 48
learning_rate: 0.001
Score: 0.887666642665863

Trial 0 summary
Hyperparameters:
conv_1_filter: 80
con_1_kernal: 3
dense_1_units: 48
learning_rate: 0.01
Score: 0.878000020980835

Trial 4 summary
Hyperparameters:
conv_1_filter: 48
con_1_kernal: 7
dense_1_units: 96
learning_rate: 0.01
Score: 0.8306666612625122


In [ ]:
best_model = tuner_search.get_best_models(num_models=1)[0]

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [ ]:
best_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 22, 22, 64)     │         3,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 16, 16, 64)     │       200,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     2,097,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,302,538 (8.78 MB)

 Trainable params: 2,302,538 (8.78 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
best_model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch= 2)

Epoch 3/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9003 - loss: 0.2656 - val_accuracy: 0.9002 - val_loss: 0.2740
Epoch 4/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9194 - loss: 0.2174 - val_accuracy: 0.9062 - val_loss: 0.2608
Epoch 5/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9310 - loss: 0.1819 - val_accuracy: 0.8983 - val_loss: 0.2786
Epoch 6/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9427 - loss: 0.1536 - val_accuracy: 0.9057 - val_loss: 0.2810
Epoch 7/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9525 - loss: 0.1243 - val_accuracy: 0.9020 - val_loss: 0.2980
Epoch 8/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9606 - loss: 0.1045 - val_accuracy: 0.9028 - val_loss: 0.3497
Epoch 9/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9661 - loss: 0.0924 - val_accuracy: 0.9068 - val_loss: 0.3812
Epoch 10/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9720 - loss: 0.0742 